In [ ]:
import sys
import os

# Path to the directory containing GABC.py
custom_path = './../lib/'
# Add the custom path to the system path
sys.path.append(custom_path)
sys.path.append("./../")
import numpy as np
import time
from scipy.optimize import differential_evolution, minimize
import matplotlib.pyplot as plt
from pyminion import *

In [ ]:
function_name_to_function = {
    "sphere": sphere,
    "rosenbrock": rosenbrock,
    "rastrigin": rastrigin,
    "drop_wave":  drop_wave, 
    "griewank": griewank,
    "ackley": ackley,
    "zakharov": zakharov,
    "goldstein_price" : goldstein_price,
    "michalewicz": michalewicz,
    "easom" : easom,
    "levy": levy,
    "dixon_price": dixon_price,
    "bent_cigar": bent_cigar,
    "discus": discus,
    "weierstrass": weierstrass,
    "happy_cat": happy_cat,
    "hgbat": hgbat,
    "hcf": hcf,
    "grie_rosen": grie_rosen,
    "escaffer6": escaffer6,
    "hybrid_composition1": hybrid_composition1,
    "hybrid_composition2": hybrid_composition2,
    "hybrid_composition3": hybrid_composition3,
    "step": step,
    "quartic": quartic,
    "schaffer2": schaffer2,
    "brown": brown,
    "exponential": exponential,
    "styblinski_tang": styblinski_tang,
    "sum_squares": sum_squares
}

# Map function implementations to function names
function_to_function_name = {v: k for k, v in function_name_to_function.items()}

# Function evaluation counter
class FunctionEvaluator:
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += 1
        ret = self.func(np.array([X]))[0]
        #print(X.shape, ret.shape)
        return ret
    
class VectorizedEvaluator : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        self.n_calls += X.shape[0]
        ret = self.func(np.array(X))
        return ret
    
class PyminionFunc : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X, data=None):
        X= np.array(X)
        self.n_calls += X.shape[0]
        return  self.func(X)
    
class VectorizedEvaluatorDE : 
    def __init__(self, func):
        self.func = func
        self.n_calls = 0

    def __call__(self, X):
        X = np.array(X).T
        self.n_calls += X.shape[0]
        return self.func(X)


In [ ]:
results = []
def test_optimization(func, bounds, dimension, func_name, Nmaxeval):
    result = {}
    result['Dimensions'] = dimension
    result['Function'] = func_name
    # Initialize bounds
    bounds_list = [bounds] * dimension
    # Create wrapped function evaluator
    evaluator = FunctionEvaluator(func)
    vecEvaluator = VectorizedEvaluator(func)
    vecEvaluatorDE = VectorizedEvaluatorDE(func)
    pyminionFunc = PyminionFunc(func)
     # Lshade Optimization
    popsize=20+round(dimension/2)
    shade = MFADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), memeorySize=popsize, callback=None, boundStrategy="reflect-random", seed=None)
    res = shade.optimize()
    result['MFADE'] = res.fun

    jade = FADE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, 
                 strategy="current_to_pbest1bin", relTol=0.0, minPopSize=max(round(dimension/2), 10), c=0.5, callback=None, boundStrategy="reflect-random", seed=None)
    res = jade.optimize()
    result['FADE'] = res.fun

    gwo = GWO_DE (pyminionFunc, bounds_list, data=None, x0=None, population_size=popsize, maxevals=Nmaxeval, F=0.5, CR=0.7, elimination_prob=0.1, relTol=0.0001, callback=None, boundStrategy="reflect-random", seed=None)
    res = gwo.optimize()
    result['GWO_DE++'] = res.fun

    # Differential Evolution (DE)
    #vecEvaluatorDE.n_calls = 0
    #psize = round(max(int(150/dimension), 1)*dimension)
    ##de_result = differential_evolution(vecEvaluatorDE, bounds_list, popsize=10, strategy='best1bin',
    #                                     maxiter=int(Nmaxeval/(10*dimension)), vectorized=True, updating="deferred", disp=False,polish=False)
    #result['Scipy DE'] = de_result.fun

    gwo_de_optimizer = GWO_DE_(objective_function=func, bounds=bounds_list, population_size=popsize, maxevals=Nmaxeval)
    best_position, best_score = gwo_de_optimizer.optimize()
    result['GWO_DE'] = best_score

    results.append(result)
    print(result)
    

    if False:
      plt.figure(figsize=(4,3))
      plt.plot(jade.muCR, label ="JADE CR")
      plt.plot(jade.muF, label="JADE F")
      plt.legend()
      plt.show()
  

      plt.figure(figsize=(4,3))
      plt.plot(shade.muCR, label ="SHADE CR")
      plt.plot(shade.muF, label="SHADE F")
      plt.legend()
      plt.show()
     
      #plt.plot(np.array(jde_optimizer.history)[:, 1], label="Fitness JDE")
      #plt.plot(np.array(shade.history)[:,1], label="Fitness SHADE")
      #plt.plot([h.fun for h in jde_.history], label="Fitness JADE C++")
      #plt.plot(fitnesses_, label="Fitness LSHADE", color="cyan", linestyle="dashed")
      #plt.legend()
      #plt.yscale("log")
      #plt.show() 
      
Nmaxeval = 150000
dimensions = [ 30]
test = function_name_to_function #{"Any" : rastrigin}
#test = {"Any" : hybrid_composition1}
for dim in dimensions:
    i=0
    for funcname, func in test.items() : #function_name_to_function.items():
        test_optimization(func, (-10, 10), dim, funcname, Nmaxeval)
        i=i+1 
        #if i>15: break 

# Results output
import pandas as pd
pd.set_option('display.float_format', '{:.12f}'.format)
results_df = pd.DataFrame(results)
results_df